In [3]:
import os
import json
import cv2
import numpy as np
from pathlib import Path

def process_images_and_labels(image_folder, label_folder, output_folder):
    # Create output folder if it doesn't exist, or clear its contents if it does
    output_path = Path(output_folder)
    if output_path.exists():
        for item in output_path.iterdir():
            if item.is_file():
                item.unlink()
    else:
        output_path.mkdir(parents=True)

    # Get sorted list of image and label files
    image_files = sorted([f for f in os.listdir(image_folder) if f.endswith(('.png', '.jpg', '.jpeg'))])
    label_files = sorted([f for f in os.listdir(label_folder) if f.endswith('.json')])

    # Process every 3rd label file
    for i, label_file in enumerate(label_files[2::3], start=0):
        if i >= len(image_files):
            print(f"Warning: No corresponding image for label {label_file}")
            continue

        # Load corresponding image
        img_file = image_files[i]
        img_path = os.path.join(image_folder, img_file)
        img = cv2.imread(img_path)

        if img is None:
            print(f"Warning: Could not read image {img_file}")
            continue

        # Print the pairing
        print(f"Processing: Image '{img_file}' with annotation '{label_file}'")

        # Load label data
        label_path = os.path.join(label_folder, label_file)
        with open(label_path, 'r') as f:
            label_data = json.load(f)

        # Draw bounding boxes
        for obj in label_data['objects']:
            bbox = obj['bbox']
            x1, y1 = map(int, bbox[0])
            x2, y2 = map(int, bbox[1])
            cv2.rectangle(img, (x1, y1), (x2, y2), (0, 0, 255), 4)  # Orange color in BGR

        # Save the image with bounding boxes
        output_file = os.path.join(output_folder, f"bbox_{img_file}")
        cv2.imwrite(output_file, img)

    print(f"Processed images saved in {output_folder}")

# Usage
# Die ersten 10 Bilder müssen zu 01, 02.... umbenannt werden
image_folder = 'runs/detect/predict'
label_folder = 'Datasets/Original/Dataset_HSKempten/annotations'
output_folder = 'runs/detect/evaluation'

process_images_and_labels(image_folder, label_folder, output_folder)


Processing: Image 'image00.jpg' with annotation 'sugarbeet_bbch12_000003.json'
Processing: Image 'image01.jpg' with annotation 'sugarbeet_bbch12_000006.json'
Processing: Image 'image02.jpg' with annotation 'sugarbeet_bbch12_000009.json'
Processing: Image 'image03.jpg' with annotation 'sugarbeet_bbch12_000012.json'
Processing: Image 'image04.jpg' with annotation 'sugarbeet_bbch12_000015.json'
Processing: Image 'image05.jpg' with annotation 'sugarbeet_bbch12_000018.json'
Processing: Image 'image06.jpg' with annotation 'sugarbeet_bbch12_000021.json'
Processing: Image 'image07.jpg' with annotation 'sugarbeet_bbch12_000024.json'
Processing: Image 'image08.jpg' with annotation 'sugarbeet_bbch12_000027.json'
Processing: Image 'image09.jpg' with annotation 'sugarbeet_bbch12_000030.json'
Processing: Image 'image10.jpg' with annotation 'sugarbeet_bbch12_000033.json'
Processing: Image 'image11.jpg' with annotation 'sugarbeet_bbch12_000036.json'
Processing: Image 'image12.jpg' with annotation 'sug

In [2]:
import matplotlib.pyplot as plt
import cv2
import json
import os
from pathlib import Path

def plot_image_with_labels(output_folder, image_name, label_folder, label_file, threshold):
    # Load the image
    img_path = os.path.join(output_folder, image_name)
    img = cv2.imread(img_path)
    if img is None:
        print(f"Error: Could not read image {image_name}")
        return

    # Convert BGR to RGB for correct color display
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # Load corresponding label file
    label_path = os.path.join(label_folder, label_file)

    if not os.path.exists(label_path):
        print(f"Error: Label file not found for {image_name}")
        return

    with open(label_path, 'r') as f:
        label_data = json.load(f)

    # Plot the image
    plt.figure(figsize=(12, 8))
    plt.imshow(img_rgb)

    small_boxes = []

    # Draw bounding boxes and find small ones
    for obj in label_data['objects']:
        bbox = obj['bbox']
        x1, y1 = map(int, bbox[0])
        x2, y2 = map(int, bbox[1])

        width = x2 - x1
        height = y2 - y1

        if width < threshold or height < threshold:
            small_boxes.append((x1, y1, x2, y2, width, height))
            rect = plt.Rectangle((x1, y1), width, height, fill=False, edgecolor='green', linewidth=2)
        else:
            rect = plt.Rectangle((x1, y1), width, height, fill=False, edgecolor='red', linewidth=2)

        plt.gca().add_patch(rect)

    plt.title(f"Image: {image_name}")
    plt.axis('off')

    # Highlight and label small bounding boxes
    for i, (x1, y1, x2, y2, width, height) in enumerate(small_boxes):
        plt.text(x1, y1-10, f"Small {i+1}: {width}x{height}", color='green', fontweight='bold', fontsize=8)

    plt.show()

    print(f"Bounding boxes below threshold {threshold}:")
    for i, (x1, y1, x2, y2, width, height) in enumerate(small_boxes):
        print(f"Box {i+1}: Width = {width}, Height = {height}")

# Usage
output_folder = 'runs/detect/evaluation'
label_folder = 'Datasets/Original/Dataset_HSKempten/annotations'
label_file = 'sugarbeet_bbch12_000147.json'
image_name = 'bbox_image48.jpg'
threshold = 15  # Adjust this value as needed

plot_image_with_labels(output_folder, image_name, label_folder, label_file, threshold)


Error: Could not read image bbox_image48.jpg
